In [0]:
import shap
import mlflow
from pyspark.sql import functions as f
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report, precision_recall_curve, average_precision_score
import seaborn as sns
from pyspark.sql.types import IntegerType
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [0]:

feature_cols_post_care = [
#"id",
 #### diagnoza ####
 'diagnoza_detail_kod', # cat IMPORTANT!
 # 'diagnoza_kod', # vzdy c50, nema smysl
 'lateralita_kod', # num
 'topografie_kod', # cat zahodit?
 'morfologie_kod', # cat zahodit?
 'grading', # numerizujeme ###
 'tnm_klasifikace_t_kod', # numerizujeme
 'tnm_klasifikace_n_kod', # numerizujeme
 'tnm_klasifikace_m_kod', # numerizujeme
 'tnm_klasifikace_metastazy_oth', # bool
 'tnm_klasifikace_metastazy_ski', # bool
 'tnm_klasifikace_metastazy_lym', # bool
 'tnm_klasifikace_metastazy_adr', # bool
 'tnm_klasifikace_metastazy_bra', # bool
 'tnm_klasifikace_metastazy_per', # bool
 'tnm_klasifikace_metastazy_hep', # bool
 'tnm_klasifikace_metastazy_ple', # bool
 'tnm_klasifikace_metastazy_oss', # bool
 'tnm_klasifikace_metastazy_mar', # bool
 'tnm_klasifikace_metastazy_pul', # bool
 'tnm_klasifikace_metastazy_xxx', # bool
 # 'tnm_klasifikace_vysledna', zahazujeme duplikace 
 'stadium', # numerizujeme
 # 'rok_dg',
 # 'mesic_dg',
 
 #### pacient ####
 'vekova_kategorie_10let_dg', # numerizujeme
 'novotvar_poradi', # num
 'novotvar_poradi_dg', # bool
 'novotvar_poradi_dg_skupina', # bool
 'novotvar_poradi_dg_maligni', # num
 'novotvar_poradi_dg_maligni_bez_c44',
 
 #### pridruzene diagnozy ####
 'DCCI', # num
 'dcci_infarkt_myokardu', # bool
 'dcci_srdecni_selhani', # bool
 'dcci_onemocneni_perifernich_cev', # bool
 'dcci_cevni_nemoci_mozku', # bool
 'dcci_demence', # bool
 'dcci_chronicke_plicni_onemocneni', # bool
 'dcci_onemocneni_pojivovych_tkani', # bool
 'dcci_vredova_onemocneni', # bool
 'dcci_mirna_onemocneni_jater', # bool
 'dcci_diabetes_bez_chronickych_komplikaci', # bool
 'dcci_diabetes_s_chronickymi_komplikacemi', # bool
 'dcci_hemiplegia_or_paraplegia', # bool
 'dcci_onemocneni_ledvin', # bool
 'dcci_nadorova_onemocneni', # bool
 'dcci_stredne_zavazne_nebo_vazne_onemocneni_jater', # bool
 'dcci_nadorova_onemocneni_s_metastazemi', # bool
 'time_indexdate_to_datum_dg', # num
 
 #### vysetreni pri primarni diagnoze ####
 'pd_ct', # bool
 'pd_petct', # bool
 'pd_mr', # bool
 'pd_scint', # bool
 'pd_spect', # bool
 'pd_rtg', # bool
 'pd_sono', # bool
 'pd_mamo', # bool
 'pd_screening', # bool
 'pd_jina', # bool

 #### Provedeni konsilia ####
 'je_mdt',
 'time_indexdate_to_mdt',
 
 ### # Kde je pacient lecen ####
 # 'je_pl', # zahodit, nelecene vyhodime z datasetu
 'je_pl_koc',
 'je_pl_roc',
 'je_pl_jinde',

 #### Delka a zpozdeni zahajeni prim lecby ####
 'time_datum_dg_to_zahajeni_pl',
 'pl_delka',
 
 #### Uzite modality lecby #### 
 'je_pl_oper',
 'je_pl_radio',
 'je_pl_target',
 'je_pl_chemo',
 'je_pl_hormo',
 'je_pl_imuno',
 'pl_pocet_leceb',

#### Poradi modalit lecby #### 
#  'pl_typ_lecby_1',
#  'pl_kod_lecby_1',
#  'pl_typ_lecby_2',
#  'pl_kod_lecby_2',
#  'pl_typ_lecby_3',
#  'pl_kod_lecby_3',
#  'pl_typ_lecby_4',
#  'pl_kod_lecby_4',
#  'pl_typ_lecby_5',
#  'pl_kod_lecby_5',
#  'pl_typ_lecby_6',
#  'pl_kod_lecby_6',
#  'pl_typ_lecby_7',
#  'pl_kod_lecby_7',
#  'pl_typ_lecby_8',
#  'pl_kod_lecby_8',
#  'pl_typ_lecby_9',
#  'pl_kod_lecby_9',
#  'pl_typ_lecby_10',
#  'pl_kod_lecby_10',
 
 #### Hospitalizace #### 
 'pl_pocet_hp',
 'pl_hp_los',
 'pl_hp_od_stan',
 'pl_hp_od_int',
 'pl_hp_drg_prvni',
 'pl_hp_drg_posledni',

 #### Vysetreni pri / po konci pl ####
 'pl_ct',
 'pl_petct',
 'pl_mr',
 'pl_scint',
 'pl_spect',
 'pl_rtg',
 'pl_sono',
 'pl_mamo',
 'pl_jina',

 #### Disperzni vysetreni ####
 # nedavat, po skonceni pl
#  'je_disp',
#  'je_disp_prakt',
#  'je_disp_onk',
#  'je_disp_int',
#  'je_disp_chir',
#  'je_disp_gyn',

#### Navstevy a body #### #nemuzem az do nl
 # 'amb_onk_navstevy', 
#  'amb_onk_body_rok1',
#  'amb_onk_body_rok2',
#  'amb_onk_body_rok3',
#  'amb_onk_body_rok4',
#  'amb_onk_body_rok5',
#  'amb_int_navstevy',
#  'amb_int_body_rok1',
#  'amb_int_body_rok2',
#  'amb_int_body_rok3',
#  'amb_int_body_rok4',
#  'amb_int_body_rok5',
#  'amb_chir_navstevy',
#  'amb_chir_body_rok1',
#  'amb_chir_body_rok2',
#  'amb_chir_body_rok3',
#  'amb_chir_body_rok4',
#  'amb_chir_body_rok5',
#  'amb_gyn_navstevy',
#  'amb_gyn_body_rok1',
#  'amb_gyn_body_rok2',
#  'amb_gyn_body_rok3',
#  'amb_gyn_body_rok4',
#  'amb_gyn_body_rok5',
#  'amb_celkem_navstevy',
#  'amb_celkem_body_rok1',
#  'amb_celkem_body_rok2',
#  'amb_celkem_body_rok3',
#  'amb_celkem_body_rok4',
#  'amb_celkem_body_rok5',
 
 #### Naslenda lecba ####
#  'je_nl',
#  'je_nl_koc',
#  'je_nl_roc',
#  'je_nl_jinde',
#  'time_datum_dg_to_zahajeni_nl',
#  'nl_delka',
#  'je_nl_oper',
#  'je_nl_radio',
#  'je_nl_target',
#  'je_nl_chemo',
#  'je_nl_hormo',
#  'je_nl_imuno',
#  'nl_pocet_leceb',
#  'nl_typ_lecby_1',
#  'nl_kod_lecby_1',
#  'nl_typ_lecby_2',
#  'nl_kod_lecby_2',
#  'nl_typ_lecby_3',
#  'nl_kod_lecby_3',
#  'nl_typ_lecby_4',
#  'nl_kod_lecby_4',
#  'nl_typ_lecby_5',
#  'nl_kod_lecby_5',
#  'nl_typ_lecby_6',
#  'nl_kod_lecby_6',
#  'nl_typ_lecby_7',
#  'nl_kod_lecby_7',
#  'nl_typ_lecby_8',
#  'nl_kod_lecby_8',
#  'nl_typ_lecby_9',
#  'nl_kod_lecby_9',
#  'nl_typ_lecby_10',
#  'nl_kod_lecby_10',
#  'nl_pocet_hp',
#  'nl_hp_los',
#  'nl_hp_od_stan',
#  'nl_hp_od_int',
#  'nl_hp_drg_prvni',
#  'nl_hp_drg_posledni',

#### Souhrn a smrti ####
#  'souhrn_diagnoz',
#  'nl_only_hormo',
#  'nl_others',
"dead_5_years_bc"
]

In [0]:
#df = (spark.read.table("rakathon_hackathon.rekurze.rekurze_death_initial_set_censured")
df = (spark.read.table("rakathon_hackathon.rekurze.rekurze_death_initial_set_censured_oprava")
      .withColumn("stadium", f.when(f.col("stadium").isin("1", "2", "3", "4"), f.col("stadium")).otherwise(None))
      .withColumn("stadium", f.col("stadium").cast(IntegerType()))
      .select(*feature_cols_post_care)
      .filter(f.col("je_pl")==1)
      .filter(f.col("dead_5_years_bc").isNotNull()))

#display(df.filter(f.col("je_pl")==1).count())
#display(df.filter(f.col("pl_delka").isNotNull()).count())
#display(df.count())
display(df)

Databricks visualization. Run in Databricks to view.

In [0]:
# 1. Convert Spark DF to pandas
df_pd = df.toPandas()

# 2. Define X, y
target_col = "dead_5_years_bc"
X = df_pd.drop(columns=[target_col])
y = df_pd[target_col]

# 3. Identify categorical columns
categorical_cols = X.select_dtypes(include=["object"]).columns.tolist()
numerical_cols = X.select_dtypes(exclude=["object"]).columns.tolist()

# 4. Define preprocessor (OneHotEncoder only for categorical)
preprocessor = ColumnTransformer(
    transformers=[
        #("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ("num", "passthrough", numerical_cols)
    ]
)

# 5. Define pipeline with preprocessor + model
model_params = {
    "use_label_encoder":True,
    "eval_metric":"auc",
    "n_estimators":1000,
    "max_depth":15,
    "learning_rate":0.01,
    "subsample":0.8
}

pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", XGBClassifier(
        **model_params
    ))
])

# 6. Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 7. MLflow tracking
with mlflow.start_run(run_name="rackathon_xgboost_pipeline"):
    # Log parameters manually
    mlflow.log_params(model_params)

    # Fit pipeline
    pipeline.fit(X_train, y_train)

    # Predict
    y_pred = pipeline.predict(X_test)
    y_pred_proba = pipeline.predict_proba(X_test)

    # Metrics
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    

    # Log metrics
    mlflow.log_metrics({
        "accuracy": accuracy,
        "f1_score": f1,
        "recall": recall,
        "precision":precision,
    })

    # Log pipeline model (includes preprocessing)
    mlflow.sklearn.log_model(pipeline, artifact_path="xgb_pipeline_model")

    print(f"✅ Accuracy: {accuracy:.4f} | F1: {f1:.4f} | Recall: {recall:.4f} | Precision: {precision:.4f}")

In [0]:
y_pred_proba_b = [x[1] for x in y_pred_proba]
df_test = pd.DataFrame({"y_test_gt":y_test, "y_test_pred_proba":y_pred_proba_b})

sns.displot(
    data=df_test,
    x="y_test_pred_proba",
    hue="y_test_gt",
    #kind="kde",
    common_norm=False,
    stat="density",
    height=7,
    aspect=2
)

plt.show()

In [0]:
pipeline.steps[0][1]

In [0]:
# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1])
disp.plot(cmap='Blues', values_format='d')
plt.title("Confusion Matrix")
plt.show()


In [0]:
print(classification_report(y_test, y_pred, digits=4))

In [0]:
# Predict probabilities for class 1
y_proba = pipeline.predict_proba(X_test)[:, 1]

# Compute precision-recall pairs and AUPRC
precision, recall, thresholds = precision_recall_curve(y_test, y_proba)
auprc = average_precision_score(y_test, y_proba)

# Plot PR curve
plt.figure(figsize=(7, 5))
plt.plot(recall, precision, label=f"AUPRC = {auprc:.4f}")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve")
plt.legend()
plt.grid(True)
plt.show()


In [0]:
X_train

In [0]:
X_train["target"] = y_train
X_train.head()

In [0]:
X_train["target"].value_counts()

## shapy

In [0]:
# 1. Extract model and preprocessor from pipeline
model = pipeline.named_steps["classifier"]
preprocessor = pipeline.named_steps["preprocessor"]

# 2. Transform X_test using the fitted preprocessor
X_test_transformed = preprocessor.transform(X_test)

# 3. Get feature names from preprocessor
feature_names = preprocessor.get_feature_names_out()

# 4. Convert transformed data to DataFrame with correct feature names
X_test_transformed_df = pd.DataFrame(
    X_test_transformed.toarray() if hasattr(X_test_transformed, "toarray") else X_test_transformed,
    columns=feature_names
)

# 5. Create SHAP explainer for the XGBoost model
explainer = shap.Explainer(model)

# 6. Compute SHAP values
shap_values = explainer(X_test_transformed_df)

# 7. Plot SHAP summary with feature names
shap.initjs()
shap.summary_plot(shap_values, X_test_transformed_df)

In [0]:
# Index of the highest probability
max_index = np.argmax(y_proba)
min_index = np.argmin(y_proba)

# Value at that index
max_proba = y_proba[max_index]
min_proba = y_proba[min_index]

print(f"🔥 Highest predicted probability is {max_proba:.4f} at index {max_index}")
print(f"🔥 Highest predicted probability is {min_proba:.4f} at index {min_index}")

In [0]:
# Step 1: get preprocessed test data
X_test_transformed = preprocessor.transform(X_test)
feature_names = preprocessor.get_feature_names_out()

# Step 2: convert to DataFrame for explainability
X_test_transformed_df = pd.DataFrame(
    X_test_transformed.toarray() if hasattr(X_test_transformed, "toarray") else X_test_transformed,
    columns=feature_names
)

def explain_prediction(i, model, explainer, shap_values, X_test_transformed_df, original_X_test):
    print(f"\n🔍 Explanation for Row #{i}")
    proba = pipeline.predict_proba(original_X_test.iloc[[i]])[0, 1]
    print(f"🔮 Predicted Probability of relaps in 5 years: {proba:.4f}\n")
    
    shap.initjs()
    return shap.force_plot(
        explainer.expected_value,
        shap_values[i].values,
        X_test_transformed_df.iloc[i],
        matplotlib=True
    )


# Example usage:
explain_prediction(14, model, explainer, shap_values, X_test_transformed_df, X_test)



# inference df

In [0]:
# 1. Read the original row by ID
original_df = (
    spark.read.table("rakathon_hackathon.rekurze.rekurze_death_initial_set_censured")
    .filter(f.col("id") == 213730)
)

# 2. Modify the stadium column from 1 to 3
modified_df = original_df.withColumn("je_pl_radio", f.lit(0)) # je_pl_koc

# 3. Union original + modified row (optional)
inference_df = original_df.unionByName(modified_df).select(*feature_cols_post_care)

# 4. Show both rows (for verification)
display(inference_df)

In [0]:
# Let's assume your modified Spark DataFrame is called `result_df`
pandas_input = inference_df.toPandas()
X_new = pandas_input.drop(columns=[target_col,"id"], errors="ignore")  # or add 'relaps_5_years' if it's present

In [0]:
# Predict class
pred_class = pipeline.predict(X_new)

# Predict probability (positive class, e.g. relaps in 5 years)
pred_proba = pipeline.predict_proba(X_new)[:, 1]


display(pred_proba)

In [0]:
# Step 1: get preprocessed test data
X_test_transformed = preprocessor.transform(X_new)
feature_names = preprocessor.get_feature_names_out()

# Step 2: convert to DataFrame for explainability
X_test_transformed_df = pd.DataFrame(
    X_test_transformed.toarray() if hasattr(X_test_transformed, "toarray") else X_test_transformed,
    columns=feature_names
)

def explain_prediction(i, model, explainer, shap_values, X_test_transformed_df, original_X_test):
    print(f"\n🔍 Explanation for Row #{i}")
    proba = pipeline.predict_proba(original_X_test.iloc[[i]])[0, 1]
    print(f"🔮 Predicted Probability of relaps in 5 years: {proba:.4f}\n")
    
    shap.initjs()
    return shap.force_plot(
        explainer.expected_value,
        shap_values[i].values,
        X_test_transformed_df.iloc[i],
        matplotlib=True
    )


# Example usage:
explain_prediction(0, model, explainer, shap_values, X_test_transformed_df, X_new)

In [0]:
explain_prediction(1, model, explainer, shap_values, X_test_transformed_df, X_new) # Tomuhle přidat hormono

## calibration

In [0]:
from sklearn.calibration import calibration_curve
# 1. Get predicted probabilities for positive class (relaps=1)
y_proba = pipeline.predict_proba(X_test)[:, 1]

# 2. Compute calibration curve
prob_true, prob_pred = calibration_curve(y_test, y_proba, n_bins=10, strategy="quantile")

# 3. Plot calibration curve
plt.figure(figsize=(8, 6))
plt.plot(prob_pred, prob_true, marker='o', label="XGBoost Pipeline")
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label="Perfect Calibration")
plt.xlabel("Mean Predicted Probability")
plt.ylabel("True Fraction of Positives")
plt.title("Calibration Curve (Reliability Diagram)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# 4. Optional: plot histogram of predicted probabilities
plt.figure(figsize=(8, 3))
plt.hist(y_proba, bins=20, edgecolor='k')
plt.title("Histogram of Predicted Probabilities")
plt.xlabel("Predicted Probability")
plt.ylabel("Frequency")
plt.grid(True)
plt.tight_layout()
plt.show()